---
layout: post

title: 백준 문제 중 15683번 with python, 구현, 감시

categort: Algorithms
---

# 백준 문제 중 15683번

<a href = 'https://www.acmicpc.net/problem/15683'>https://www.acmicpc.net/problem/15683</a>

### 문제

스타트링크의 사무실은 1×1크기의 정사각형으로 나누어져 있는 N×M 크기의 직사각형으로 나타낼 수 있다. 사무실에는 총 K개의 CCTV가 설치되어져 있는데, CCTV는 5가지 종류가 있다. 각 CCTV가 감시할 수 있는 방법은 다음과 같다.

<div display = 'flex'>
  <img src = 'https://onlinejudgeimages.s3-ap-northeast-1.amazonaws.com/problem/15683/1.png' width = '15%' >
  <img src = 'https://onlinejudgeimages.s3-ap-northeast-1.amazonaws.com/problem/15683/2.png' width = '20%'>
  <img src = 'https://onlinejudgeimages.s3-ap-northeast-1.amazonaws.com/problem/15683/3.png' width = '15%' >
  <img src = 'https://onlinejudgeimages.s3-ap-northeast-1.amazonaws.com/problem/15683/4.png' width = '20%' >
  <img src = 'https://onlinejudgeimages.s3-ap-northeast-1.amazonaws.com/problem/15683/5.png' width = '20%'>
</div>

1번 CCTV는 한 쪽 방향만 감시할 수 있다. 2번과 3번은 두 방향을 감시할 수 있는데, 2번은 감시하는 방향이 서로 반대방향이어야 하고, 3번은 직각 방향이어야 한다. 4번은 세 방향, 5번은 네 방향을 감시할 수 있다.

CCTV는 감시할 수 있는 방향에 있는 칸 전체를 감시할 수 있다. 사무실에는 벽이 있는데, CCTV는 벽을 통과할 수 없다. CCTV가 감시할 수 없는 영역은 사각지대라고 한다.

CCTV는 회전시킬 수 있는데, 회전은 항상 90도 방향으로 해야 하며, 감시하려고 하는 방향이 가로 또는 세로 방향이어야 한다.

### 입력

첫째 줄에 사무실의 세로 크기 N과 가로 크기 M이 주어진다. (1 ≤ N, M ≤ 8)

둘째 줄부터 N개의 줄에는 사무실 각 칸의 정보가 주어진다. 0은 빈 칸, 6은 벽, 1~5는 CCTV를 나타내고, 문제에서 설명한 CCTV의 종류이다. 

CCTV의 최대 개수는 8개를 넘지 않는다.

### 출력

첫째 줄에 사각 지대의 최소 크기를 출력한다.

---

 CCTV의 종류에 따라 동작을 나누는게 문제의 핵심이었다. 모든 CCTV에 대해서 모든 경우를 탐색해 사각지대가 최소일땔르 구하면 된다.

CCTV를 종류에 따른 동작을 딕셔너리에 선언하고, 종류에 따라 DFS를 실행 하게 코드를 구성했다.

코드의 흐름은 이렇다.
1. CCTV가 설치된 좌표와 종류를 points 리스트에 담는다.
2. CCTV 종류에 따라 동작을 나타낼 딕셔너리 cctv를 선언한다.
3. 최악의 경우의 사각구역의 수를 ans에 초기화한다.
4. 할당된 방향으로 인덱스 범위를 벗어나지않고 벽을 만나기 전까지 구역을 #으로 바꿔줄 함수 mark_observable을 선언한다.
5. 재귀적으로 DFS로 모든 탐색을 진행할 dfs를 선언한다.

In [57]:
from copy import deepcopy
from typing import List

n, m = map(int, input().split())
board = [list(input().split()) for _ in range(n)]

 # CCTV가 설치된 모든 좌표와 종류를 points 리스트에 저장
points = []
for i in range(n):
    for j in range(m):
        if board[i][j] in '12345':
            points.append([i,j, board[i][j]])

 # CCTV 종류에 따라 동작을 구별할 딕셔너리 선언
cctv = {
    '1': [
        [[0, 1]], [[1, 0]], [[0, -1]], [[-1, 0]] 
    ], 
    '2': [
        [[0, 1], [0, -1]], [[1, 0], [-1, 0]]
    ],
    '3': [
        [[-1, 0], [0, 1]], [[1, 0], [0, 1]], [[1, 0], [0, -1]], [[-1, 0], [0, -1]],
    ],
    '4': [
        [[-1, 0], [0, 1], [1, 0]], [[1, 0], [0, 1], [0, -1]], 
        [[-1, 0], [1, 0], [0, -1]], [[-1, 0], [0, -1], [0, 1]],
    ],
    '5':[
       [[1, 0], [0, 1], [-1, 0], [0, -1]]
    ]
}

 # CCTV가 하나도 없을때의 사각구역으로 ans를 초기화
ans = n*m

 # 할당된 dir들로 감시가능 구역을 모두 #으로 바꿔줄 함수 선언
def mark_observable(sub_board:List[List[str]], r:int, c:int, dir:List[List[int]])->List[List[str]]:

    for dr,dc in dir:
        nr, nc = r + dr, c + dc
        while 0 <= nr < n and 0 <= nc < m and sub_board[nr][nc] != '6':
            sub_board[nr][nc] = '#'
            nr, nc = nr + dr, nc + dc
    return sub_board
    
          
 # 재귀적으로 완전탐색을 실행할 DFS 선언
def dfs(depth:int, board:List[List[str]])->List[List[str]]:

    global ans

    if depth > 8:
        return
        
    if len(points) == depth:
        cnt = 0
        for b in board: cnt += b.count('0')
        ans = min(ans, cnt)
        return

    row, col, kind = points[depth]
    candidates = cctv[kind]
     # candidates 안에 있는 모든 동작에 대해 경우를 탐색함
    for candidate in candidates:
        tmp = deepcopy(board)
        sub_board = mark_observable(tmp, row, col, candidate)
        dfs(depth + 1, sub_board)
        

dfs(0, board)
print(ans)

3 7
4 0 0 0 0 0 0
0 0 0 2 0 0 0
0 0 0 0 0 0 4
0
